このNotebookでは簡単なサンプルコードを通じて、機械学習の一連の処理
1. ファイル読み込み
2. データの前処理
3. 学習モデルの作成
4. 学習
5. 学習済みモデルの評価
6. 学習済みモデルで判定
7. 提出用ファイルの作成  
をみていきます。

# 1. ファイル読み込み 〜 2.データの前処理

## train.csvファイルの読み込みコード

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

root_dir = "/kaggle/input/mj1-anomaly-images-detection-challenge/"
train_csv_filepath = root_dir + "train.csv"

# ファイルの読み込み
train_df = pd.read_csv(train_csv_filepath)

# 出力して中身を確認
train_df

## 画像の読み込み

In [ ]:
resize_w = 256
resize_h = 256
channel = 3

import cv2
# 画像が大きいと計算が遅いため、リサイズ縮小
def resize(tmp_image):
    return cv2.resize(tmp_image , (resize_h, resize_w))

# 4次元配列化()　
def to_4d(tmp_image):
    return tmp_image.reshape(1, resize_h, resize_w, channel)
    

# 256段階の色調を0.0~1.0にする
def normalize(tmp_image):
    return tmp_image / 255.0

# 画像の前処理付きロード
def load_preprocessed_image(image_filepath):
    tmp_image = cv2.imread(image_filepath)
    tmp_image = resize(tmp_image)
    tmp_image = normalize(tmp_image)
    tmp_image = to_4d(tmp_image)
    return tmp_image

In [ ]:
images = None
for fn in train_df['filename']:
    image_filepath = root_dir + 'train/' + fn
    tmp_image = load_preprocessed_image(image_filepath)
    if (images is None):
        images = tmp_image
    else:
        images = np.vstack((images, tmp_image))

# 確認
images.shape

## ラベル

In [ ]:
from keras.utils import np_utils

anomaly_flags = np.array([flag for flag in train_df['anomaly']])

# 確認
anomaly_flags = np_utils.to_categorical(anomaly_flags, 2)
anomaly_flags.shape


# 3.学習モデルの作成

In [ ]:
import numpy as np
#データの読み込みと前処理

from keras.datasets import mnist
#kerasでCNN構築
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

 
'''
CNNの構築
'''
 
model = Sequential()
 
model.add(Conv2D(filters=10, kernel_size=(4,4), padding='same', input_shape=(256, 256, 3), activation='relu'))
model.add(Conv2D(filters=10, kernel_size=(3,3), padding='same', input_shape=(64, 64, 8), activation='relu'))
model.add(Conv2D(filters=10, kernel_size=(2,2), padding='same', input_shape=(16, 16, 16), activation='relu'))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0003), metrics=['accuracy'])



## データ分割

学習(train)と、評価(validate、あるいはtest)用にデータを分けます。  
・学習に使ったデータは判定が簡単ですが、学習に使ってない未知のデータは判定が難しく精度に大きな差があります。
・実際に判定するデータは、未知のデータばかりなので、未知のデータで性能を測ることが一般的です。

In [ ]:
X_test = images[:100]
y_test = anomaly_flags[:100]
X_train = images[100:]
y_train = anomaly_flags[100:]

# 4. 学習処理

機械学習ライブラリの、fit()関数を呼ぶと学習を開始します。

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=16,
                    verbose=1,
                    validation_data=(X_test, y_test))

# 5.学習済みモデルの評価

In [ ]:

train_score = model.evaluate(X_train, y_train, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)


print('Train Loss:{0:.3f}'.format(train_score[0]))
print('Train accuracy:{0:.3}'.format(train_score[1]))
print('Test Loss:{0:.3f}'.format(test_score[0]))
print('Test accuracy:{0:.3}'.format(test_score[1]))

# 6. 学習済みモデルで判定

## 判定用ファイルの読み込み

In [ ]:
import glob
from pathlib import Path

test_images = None
test_filenames = None
for test_filepath in glob.glob('/kaggle/input/mj1-anomaly-images-detection-challenge/test/*.png'):
    tmp_image = load_preprocessed_image(test_filepath)
    if (test_images is None):
        test_images = tmp_image
        test_filenames = [Path(test_filepath).name]
    else:
        test_images = np.vstack((test_images, tmp_image))
        test_filenames.append(Path(test_filepath).name)

test_images.shape

In [ ]:
test_filenames[:5]

## 判定
判定は、predict()関数で行います。

In [ ]:
result_predict = model.predict(test_images)
result_predict = np.argmax(result_predict, axis=1)
result_predict

# 7.提出用ファイルの作成

kaggleコンペに提出する形式のcsvを作成します。  

計算が終了するとNotebookのデータフォルダ/Outputの下にcsvファイルが作成されます。  

In [ ]:
submit_filepath = "/kaggle/input/mj1-anomaly-images-detection-challenge/sample_submit.csv"
submit_df = pd.read_csv(submit_filepath, index_col=0)
submit_df

In [ ]:
for i, filename in enumerate(test_filenames):
    submit_df.loc[filename, 'Predicted'] = result_predict[i]
submit_df

In [ ]:
submit_df.to_csv('result_submit.csv')